In [12]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from datetime import date

In [13]:
matplotlib.use("QtAgg")

In [14]:
data = pd.read_csv('small_panhandle_data.csv')

In [15]:
data=data.drop(columns = ['dateF'])

In [16]:
#I thought I would need year/month/day format for other methods to work

In [17]:
spring = data[data.date.between("03/20/2024","06/19/2024")]

In [18]:
summer = data[data.date.between("06/20/2024","09/21/2024")]

In [19]:
fall = data[data.date.between("09/22/2024","12/20/2024")]

In [20]:
#Winter is 0, spring is 1, summer is 2, fall is 3 in the 'season' column

In [21]:
data['season'] = 0

In [22]:
#This creates the boolean mask for and assigns each season value to 'season' column.
#I think there's a way to make this shorter and more efficient but I don't nokw how

In [23]:
springmask = data['date'].between("03/20/2024","06/19/2024")

In [24]:
data.loc[springmask,'season']= 1

In [25]:
summermask = data['date'].between("06/20/2024","09/21/2024")

In [26]:
data.loc[summermask,'season']= 2

In [27]:
fallmask = data['date'].between("09/22/2024","12/20/2024")

In [28]:
data.loc[fallmask,'season']= 3

In [29]:
def differentiate(a,b):
    for i in range (1,len(a)):
        b[i]=float(a[i]-a[i-1])

In [30]:
data['differential']=0

In [31]:
differentiate(data['price'],data['differential'])

C:\Users\runni\AppData\Local\Temp\ipykernel_13480\1524461724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b[i]=float(a[i]-a[i-1])
C:\Users\runni\AppData\Local\Temp\ipykernel_13480\1524461724.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7400000000000002' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  b[i]=float(a[i]-a[i-1])


In [32]:
data

,date,hour,price,season,differential
0,01/01/2024,1,14.19,0,0.00
1,01/01/2024,1,14.93,0,0.74
2,01/01/2024,1,15.93,0,1.00
3,01/01/2024,1,16.05,0,0.12
4,01/01/2024,2,15.84,0,-0.21
...,...,...,...,...,...
35131,12/31/2024,23,22.37,0,0.11
35132,12/31/2024,24,21.99,0,-0.38
35133,12/31/2024,24,20.60,0,-1.39
35134,12/31/2024,24,19.78,0,-0.82


In [33]:
data['movingavg']=data['price'].rolling(window = 96, min_periods = 1).median()

In [35]:
class Bot:
    def __init__(self, balance, storage):
        self.balance = balance 
        self.storage = storage
        self.trades = 0
        
    def buy(self, price):
        if price < self.balance:
            self.balance -= price
            self.storage += 1

    def sell(self, price):
        if self.storage > 0:
            self.balance += price
            self.storage -= 1
            self.trades += 1

    def summary(self):
        print("Balance: " + str(self.balance))
        print("Storage: " + str(self.storage))
        print("Trades made: " + str(self.trades))
        print("Income per trade: " + str(self.balance/self.trades))
        print("Estimated value of storage at $20/MWh: " + str(self.storage*20))
            
class LossyBot(Bot):
    def sell(self, price):
        if self.storage > 0:
            self.balance += price *.8
            self.storage -= 1
            self.trades += 1
        

In [66]:
#Buys if price less than average, sells if more

In [37]:
def SimpleTrades(bot, market):
    for index, row in market.iterrows():
        price = row['price']
        avg = row['movingavg']
        if price<avg:
            bot.buy(price)
        elif price>avg:
            bot.sell(price)

In [68]:
#This shifts the benchmark for when to buy an arbitary amount below the average. The sell threshold coefficient is 1.25 times
#the buy threshold to reflect 80% round trip efficiency

In [70]:
def ThoughtfulTrades(bot, market):
    for index, row in market.iterrows():
        price = row['price']
        avg = row['movingavg']
        if price<avg * .9:
            bot.buy(price)
        elif price>avg * 1.125:
            bot.sell(price)

In [72]:
january=data[data.date.between("01/01/2024","01/31/2024")]

In [74]:
#Assessing performance of each bot

In [41]:
bot1 = Bot(100,0)
SimpleTrades(bot1,january)
bot1.summary()

Balance: 39080.69000000004
Storage: 193
Trades made: 1249
Income per trade: 31.289583666933577
Estimated value of storage at $20/MWh: 3860


In [42]:
bot2 = LossyBot(100,0)
SimpleTrades(bot2,january)
bot2.summary()

Balance: 27269.320000000058
Storage: 193
Trades made: 1208
Income per trade: 22.57394039735104
Estimated value of storage at $20/MWh: 3860


In [43]:
bot3 = LossyBot(100,0)
ThoughtfulTrades(bot3,january)
bot3.summary()

Balance: 30710.37000000006
Storage: 151
Trades made: 1049
Income per trade: 29.275853193517694
Estimated value of storage at $20/MWh: 3020


In [76]:
#qFinder finds a quantile at which the number of MWh price intervals that can be bought at the round-trip-efficiency fraction of 
#another price interval is maximized

In [45]:
#That seems like it's confusing, so basically, since the round-trip-efficiency is 80%, qFinder finds the point in the data
#where the most amount of data points can be "bought" at 80% of the price of other data points.
#This helps maximize the likelihood that any power bought will be sold at a profitable price

In [46]:
roundTripEfficiency = .8

In [47]:
def qFinder(val,pricedata):
    
    q = val
    k = val
    r = 1-k
    
    L = lambda l : pricedata.quantile(l)

    while (L(r) * roundTripEfficiency - L(q)) > 0 and (k<r):
        diff1 = L(r) * roundTripEfficiency - L(q)
        diff2 = L(r) * roundTripEfficiency - L(k)
        
        if diff2 < diff1 and (diff2 > 0):
            q=k
        
        k += .01
            
    return pricedata.quantile(q)   

In [48]:
q = qFinder(.4,january['price'])

In [49]:
def qTrades(bot, market, q):
    for index, row in market.iterrows():
        price = row['price']
        if price<q:
            bot.buy(price)
        elif price>q*(1/roundTripEfficiency):
            bot.sell(price)

In [50]:
bot4 = LossyBot(100,0)
qTrades(bot4,january,q)
bot4.summary()

Balance: 48626.39600000005
Storage: 177
Trades made: 1013
Income per trade: 48.002365251727596
Estimated value of storage at $20/MWh: 3540
